In [1]:
"""
We want an interctive finance chart for monthyl costs, to see affordability based on factors

Costs:
* Utilities
* Insurance
* Property Taxes

Based on factors
* Sale price
* part that is land cost
* Number of roms


For each factor, have line chart, of value vs monthly cost
"""

import pandas as pd
import altair as alt

In [2]:
import dataclasses
import typing

In [74]:
@dataclasses.dataclass
class Variable:
    label: str
    start: int
    stop: int
    step: int
    default: int
    tp: typing.Literal['$', '%', '#']
    hidden: bool = dataclasses.field(default=False)

    @property
    def axis_format(self):
        return {
            '$': '$.2s',
            '%': '%',
            '#': 's'
        }[self.tp]

def generate_plot(fn, **variables):
    sequences = {
        k: alt.sequence(v.start, v.stop, v.step, as_=k)
        for k, v in variables.items()
    }
    
    selections = {
        k: alt.selection(
            type='single',
            on='mouseover',
            nearest=True,
            fields=[k],
            init={k: v.default}
        )
        for k, v in variables.items()
    }
    current_values = {
        k: v.default if v.hidden else getattr(selections[k], k)
        for k, v in variables.items()
    }
    
    line_charts = {
        k: alt.Chart(sequences[k]).transform_calculate(
            # Take the sum of all the returned monthly costs
            monthly_cost=sum(fn(**{
                # if the input is this variable, use the data field, otherwise use the last selection for it
                inner_k: getattr(alt.datum, inner_k) if inner_k == k else current_values[inner_k]
                for inner_k in variables.keys()
            }).values())
        ).mark_line().encode(
            alt.X(field=k, type='quantitative', axis=alt.Axis(
                format=v.axis_format,
                title=v.label
            )),
            alt.Y('monthly_cost:Q', axis=alt.Axis(
                title='Monthly Cost',
                format='$.2s'
            )),
        )
        
        for k, v in variables.items()
    }
    
    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    transparent_point_charts = {
        k: alt.Chart(sequences[k]).mark_point().encode(
            alt.X(field=k, type='quantitative'),
            opacity=alt.value(0),
        ).add_selection(
            selections[k]
        ) 
        for k in variables.keys()
    }
    
    # Draw a rule at the location of the selection
    rule_charts = {
        k: alt.Chart(sequences[k]).mark_rule(
            # color='gray'
        ).encode(
            alt.X(field=k, type='quantitative'),
        ).transform_filter(
            selections[k]
        )
        for k in variables.keys()
    }
    
    
    monthly_cost_categories = fn(**current_values)
    monthly_cost = None
    for k, v in monthly_cost_categories.items():
        monthly_cost = alt.expr.if_(alt.datum.category == k, v, monthly_cost)
    
    
    base_pie_chart = alt.Chart(
        alt.InlineData([
            {"category": k}
            for k in monthly_cost_categories.keys()
        ])
    ).transform_calculate(
        cost=monthly_cost
    )


    base_pie_chart_with_theta = base_pie_chart.encode(
        theta=alt.Theta("cost:Q", stack=True),
        tooltip=['category:N', 'cost:Q']
    )

    pie_arc_chart = base_pie_chart_with_theta.mark_arc(
        innerRadius=30,
        outerRadius=120
    ).encode(
        color=alt.Color(
            "category:N",
            legend=alt.Legend(
                orient='top',
                title='Monthly cost by category'
            )
        )
    )
    pie_text_chart = base_pie_chart_with_theta.mark_text(
        radius=140,
        size=10
    ).encode(alt.Text("cost:Q", format='$.2s'))


    pie_sum_text_chart = base_pie_chart.mark_text(radius=0, size=20).encode(
        alt.Text("cost:Q", aggregate='sum', format='$.2s')
    )

    chart = alt.vconcat(
        pie_arc_chart + pie_sum_text_chart,
        alt.concat(
            *(
                alt.layer(line_charts[k], transparent_point_charts[k], rule_charts[k])
                for k, v in variables.items() if not v.hidden
            ),
            columns=2
        ).resolve_scale(
            y='shared'
        )
    )
    chart.save('index.html')
    return chart

In [82]:
TAX_RATE = 2.32
ROBBIE_PROPANE = 1135
ROBBIE_ELECTRIC = 1598
ROBBIE_HEATING_OIL = 375

generate_plot(
    lambda property_cost, interest_rate, insurance, lawyer_fees, clt_value, assessed_value, cords_wood, price_per_cord, percent_maintaince, rooms, propane, electric, oil: {
        "Home Insurance": insurance / 12 / rooms,
        "Investor Return": property_cost * interest_rate / 12 / rooms + lawyer_fees * interest_rate / 12 / rooms - clt_value * interest_rate / 12 / rooms,
        "CLT Lease Fee": alt.expr.if_(clt_value <= 0, 0, 75 / rooms),
        "Property Tax": assessed_value * TAX_RATE / 100 / 12 / rooms,
        "Wood": cords_wood *  price_per_cord / 12 / rooms,
        "Propane": propane / 12 / rooms,
        "Electric": electric / 12 / rooms,
        "Heating Oil": oil / 12 / rooms,
        "Maintaince Fund": percent_maintaince * 200 * 1000 / 12 / rooms,
    },
    rooms=Variable("Number of rooms", 1, 6, 1, 3, '#', hidden=False),
    property_cost=Variable("Purchase price", 110 * 1000, 300 * 1000, 5 * 1000, 170 * 1000, '$'),
    clt_value=Variable("Land value owned by CLT", 0, 150 * 1000, 10000, 70 * 1000, '$'),
    interest_rate=Variable("Investor interest rate", 0, 0.10, 0.01, 0.03, '%'),
    insurance=Variable("Home Insurance (annual)", 300, 800, 100, 500, '$', hidden=False),
    lawyer_fees=Variable("Lawyer fees and closing costs", 2000, 20000, 1000, 10000, '$', hidden=False),
    percent_maintaince=Variable("% of home value to set aside for maintaince", 0.01, 0.05, 0.01, 0.03, '%', hidden=False),
    assessed_value=Variable("Assessed Value", 110 * 1000, 300 * 1000, 5 * 1000, 110 * 1000, '$', hidden=False),
    cords_wood=Variable("Cords of wood per year", 2, 4, 1, 3, '#', hidden=True),
    price_per_cord=Variable("Price per cord wood", 120, 580, 10, 190, '$', hidden=False),
    propane=Variable("Propane per year", ROBBIE_PROPANE * 0.5, ROBBIE_PROPANE * 2, 10, ROBBIE_PROPANE, '$'),
    electric=Variable("Electric per year", ROBBIE_ELECTRIC * 0.5, ROBBIE_ELECTRIC * 2, 10, ROBBIE_ELECTRIC, '$'),
    oil=Variable("Heating oil per year", ROBBIE_HEATING_OIL * 0.5, ROBBIE_HEATING_OIL * 2, 10, ROBBIE_HEATING_OIL, '$')
)

alt.VConcatChart(...)